<a href="https://colab.research.google.com/github/rithulsandeep/license_plate_recognition/blob/main/ALPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install YOLOv8 via Ultralytics
!pip install ultralytics --upgrade -q

# Install OpenCV for image processing
!pip install opencv-python-headless -q

# Install EasyOCR for license plate text recognition
!pip install easyocr -q

# Install matplotlib for visualizations
!pip install matplotlib -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
weight_path = "/content/drive/MyDrive/license_plate_detector.pt"

In [5]:
import os

# Example path (replace it with your actual location)
weight_path = "/content/drive/MyDrive/license_plate_detector.pt"

# Check if the file exists
if os.path.exists(weight_path):
    print("Weights found ✅")
else:
    print("❌ Weight file not found. Check the path.")


Weights found ✅


In [6]:
from ultralytics import YOLO

# Replace this with your actual path from Step 2
# weight_path = "/content/drive/MyDrive/ALPR_Project/weights/best.pt"

# Load the YOLOv8 model
model = YOLO(weight_path)

# Confirm the model loaded correctly
print("✅ YOLOv8 model loaded successfully!")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ YOLOv8 model loaded successfully!


In [7]:
model.info()

YOLOv8n summary: 129 layers, 3,011,043 parameters, 0 gradients, 8.2 GFLOPs


(129, 3011043, 0, 8.1941504)

In [13]:
import cv2
import os
from pathlib import Path

# Set path to test images folder
img_dir = "/content/drive/MyDrive/license_plates"
output_dir = "/content/drive/MyDrive/output_plates"
os.makedirs(output_dir, exist_ok=True)

# Get list of image files
image_paths = sorted(Path(img_dir).glob("*.jpg"))  # or .png, etc.

# Loop through each image
for img_path in image_paths:
    img = cv2.imread(str(img_path))
    results = model(img)  # Run YOLOv8 inference

    boxes = results[0].boxes  # Get detections

    for i, box in enumerate(boxes):
        # Extract bounding box coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # xyxy format
        cropped_plate = img[y1:y2, x1:x2]  # Crop the plate

        # Save the cropped image
        plate_filename = f"{img_path.stem}_plate_{i}.jpg"
        plate_path = os.path.join(output_dir, plate_filename)
        cv2.imwrite(plate_path, cropped_plate)

print("✅ License plates cropped and saved successfully.")



0: 384x640 1 license_plate, 45.9ms
Speed: 16.1ms preprocess, 45.9ms inference, 340.7ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 license_plate, 35.2ms
Speed: 3.2ms preprocess, 35.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
✅ License plates cropped and saved successfully.


In [14]:
import easyocr
import os
from pathlib import Path
import pandas as pd

# Initialize EasyOCR reader (English + digits)
reader = easyocr.Reader(['en'])  # Downloads model files first time

# Paths
cropped_dir = "/content/drive/MyDrive/output_plates"
output_csv = "/content/drive/MyDrive/recognized_plates.csv"

# Store results
results_list = []

# Loop through each cropped image
for plate_img in sorted(Path(cropped_dir).glob("*.jpg")):
    result = reader.readtext(str(plate_img), detail=0, paragraph=False)

    # Join all pieces of text detected (if multiple)
    plate_text = " ".join(result).strip()

    results_list.append({
        "filename": plate_img.name,
        "plate_text": plate_text
    })

# Save to CSV
df = pd.DataFrame(results_list)
df.to_csv(output_csv, index=False)

print("✅ OCR complete. Results saved to:")
print(output_csv)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ OCR complete. Results saved to:
/content/drive/MyDrive/recognized_plates.csv


In [15]:
print("testing")

testing
